Load the test data

In [21]:
import numpy as np
import pandas as pd
import string
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
import sklearn

In [22]:
dataframe = pd.read_csv("Test.csv")

dataframe.head()
dataframe.shape

(270000, 10)

Converting it to lowercase

In [23]:
dataframe['review_text']=dataframe['review_text'].astype('str')
dataframe['review_text']=dataframe['review_text'].str.lower()
dataframe.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,6ff6f572141651ad567e301e95158a68,2418888,b3d1248da3dd5cb0c15d5e737793ca32,"[tw for book: (view spoiler)[wicca/witchcraft,...",Mon Sep 21 23:32:23 -0700 2015,Mon Sep 21 23:39:05 -0700 2015,Sat Jun 06 00:00:00 -0700 2015,NaN,0,0
1,16b60bb530dceaafde65828438bc79ca,6516450,8ac45a1ad472e40dd19245317087f009,i only found like half of them extremely inter...,Tue Apr 30 07:33:20 -0700 2013,Mon Jun 01 13:46:37 -0700 2015,Mon Jun 01 13:46:37 -0700 2015,NaN,0,0
2,7efb0e9b3ab7a9e3a5d4a0e0e4200b18,8752885,e083ab7283f9c8f0edd6f5673af348a9,the dragon's path sneaks up on you. you start ...,Thu Mar 20 04:08:20 -0700 2014,Fri Mar 21 10:04:10 -0700 2014,Fri Mar 21 10:04:10 -0700 2014,Thu Mar 20 00:00:00 -0700 2014,0,0
3,aed4f8ec7b4f51fcd5b7f2facea5a058,13484184,167e419508a6a188472e9568226da48a,3.5 stars \n this is well written and i was dr...,Thu Oct 02 11:47:41 -0700 2014,Wed Oct 15 07:55:45 -0700 2014,Sat Oct 11 00:00:00 -0700 2014,NaN,2,3
4,880dc69f3de6d55a0623d61dedd49868,13265293,03a8a7306e3ef4c1ffd95de068b58b4e,crestfallen. that's what i feel after i read t...,Sun Dec 08 11:02:59 -0800 2013,Wed Dec 11 20:04:26 -0800 2013,Wed Dec 11 00:00:00 -0800 2013,Sun Dec 08 00:00:00 -0800 2013,0,0


Expanding Contractions

In [24]:
import contractions
def expand_contractions(text):
    expanded_words = []
    for word in text.split():
       expanded_words.append(contractions.fix(word))
    return ' '.join(expanded_words)

Removal of Hashtags and mentions

In [25]:
def remove_mentions_and_tags(text):
    text = re.sub(r'@\S*', '', text)
    return re.sub(r'#\S*', '', text)

Removal of Special characters

In [26]:
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
    return re.sub(pat, '', text)

Removal of Punctutaions

In [27]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

Removal of URLs

In [28]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

Removal of HTML tags

In [29]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

Removal of stopwords

In [30]:
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

Text Cleaning

In [31]:
def text_cleaning():
    dataframe['review_text'] = dataframe['review_text'].apply(lambda text: expand_contractions(text))
    dataframe['review_text'] = dataframe['review_text'].apply(lambda text: remove_mentions_and_tags(text))
    dataframe['review_text'] = dataframe['review_text'].apply(lambda text: remove_special_characters(text))
    dataframe['review_text'] = dataframe['review_text'].apply(lambda text: remove_punctuation(text))
    dataframe['review_text'] = dataframe['review_text'].apply(lambda text: remove_urls(text))
    dataframe['review_text'] = dataframe['review_text'].apply(lambda text: remove_html(text))
    dataframe['review_text'] = dataframe['review_text'].apply(lambda text: remove_stopwords(text))

text_cleaning()
dataframe.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,6ff6f572141651ad567e301e95158a68,2418888,b3d1248da3dd5cb0c15d5e737793ca32,tw book view spoilerwiccawitchcraft minoradult...,Mon Sep 21 23:32:23 -0700 2015,Mon Sep 21 23:39:05 -0700 2015,Sat Jun 06 00:00:00 -0700 2015,NaN,0,0
1,16b60bb530dceaafde65828438bc79ca,6516450,8ac45a1ad472e40dd19245317087f009,found like half extremely interesting excellen...,Tue Apr 30 07:33:20 -0700 2013,Mon Jun 01 13:46:37 -0700 2015,Mon Jun 01 13:46:37 -0700 2015,NaN,0,0
2,7efb0e9b3ab7a9e3a5d4a0e0e4200b18,8752885,e083ab7283f9c8f0edd6f5673af348a9,dragons path sneaks start book pause middle ke...,Thu Mar 20 04:08:20 -0700 2014,Fri Mar 21 10:04:10 -0700 2014,Fri Mar 21 10:04:10 -0700 2014,Thu Mar 20 00:00:00 -0700 2014,0,0
3,aed4f8ec7b4f51fcd5b7f2facea5a058,13484184,167e419508a6a188472e9568226da48a,35 stars well written drawn story never liked ...,Thu Oct 02 11:47:41 -0700 2014,Wed Oct 15 07:55:45 -0700 2014,Sat Oct 11 00:00:00 -0700 2014,NaN,2,3
4,880dc69f3de6d55a0623d61dedd49868,13265293,03a8a7306e3ef4c1ffd95de068b58b4e,crestfallen feel read book hidden sounds aweso...,Sun Dec 08 11:02:59 -0800 2013,Wed Dec 11 20:04:26 -0800 2013,Wed Dec 11 00:00:00 -0800 2013,Sun Dec 08 00:00:00 -0800 2013,0,0


Tokenization, Lemmatization and Stemming

In [32]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def lemmatization_stemming(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return " ".join([lemmatizer.lemmatize(word) for word in stemmed_tokens])


In [33]:
def text_preprocessing():
    dataframe['review_text'] = dataframe['review_text'].apply(lambda text: lemmatization_stemming(text))

dataframe.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,6ff6f572141651ad567e301e95158a68,2418888,b3d1248da3dd5cb0c15d5e737793ca32,tw book view spoilerwiccawitchcraft minoradult...,Mon Sep 21 23:32:23 -0700 2015,Mon Sep 21 23:39:05 -0700 2015,Sat Jun 06 00:00:00 -0700 2015,NaN,0,0
1,16b60bb530dceaafde65828438bc79ca,6516450,8ac45a1ad472e40dd19245317087f009,found like half extremely interesting excellen...,Tue Apr 30 07:33:20 -0700 2013,Mon Jun 01 13:46:37 -0700 2015,Mon Jun 01 13:46:37 -0700 2015,NaN,0,0
2,7efb0e9b3ab7a9e3a5d4a0e0e4200b18,8752885,e083ab7283f9c8f0edd6f5673af348a9,dragons path sneaks start book pause middle ke...,Thu Mar 20 04:08:20 -0700 2014,Fri Mar 21 10:04:10 -0700 2014,Fri Mar 21 10:04:10 -0700 2014,Thu Mar 20 00:00:00 -0700 2014,0,0
3,aed4f8ec7b4f51fcd5b7f2facea5a058,13484184,167e419508a6a188472e9568226da48a,35 stars well written drawn story never liked ...,Thu Oct 02 11:47:41 -0700 2014,Wed Oct 15 07:55:45 -0700 2014,Sat Oct 11 00:00:00 -0700 2014,NaN,2,3
4,880dc69f3de6d55a0623d61dedd49868,13265293,03a8a7306e3ef4c1ffd95de068b58b4e,crestfallen feel read book hidden sounds aweso...,Sun Dec 08 11:02:59 -0800 2013,Wed Dec 11 20:04:26 -0800 2013,Wed Dec 11 00:00:00 -0800 2013,Sun Dec 08 00:00:00 -0800 2013,0,0


Load the training data

In [34]:
df = pd.read_csv("preprocessed_data.csv")
df.head()


,review_text,rating
0,45 stars loved book incredibly intricate wellc...,4
1,dad friend read series years friend gave gift ...,4
2,anyone desiring start exploration sm erotica s...,3
3,375 stars mal best crazy fun says things make ...,4
4,would like begin saying much appreciate amount...,5


Tfidf

In [35]:
df.drop(axis="rows", labels=df.index[df["review_text"].isna()], inplace=True)

X_train=df['review_text']
y_train=df['rating']

X_test = dataframe['review_text']
vectorizer = CountVectorizer(analyzer='word',
                             ngram_range=(1, 1),
                             max_features=500,
                             max_df=0.90,
                             min_df=5
                            )

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
X_train = tfidf_transformer.fit_transform(X_train)
X_test = tfidf_transformer.transform(X_test)
X_train = pd.DataFrame(X_train.toarray(),
             columns=vectorizer.get_feature_names_out())
X_test = pd.DataFrame(X_test.toarray(),
             columns=vectorizer.get_feature_names_out())

Standard Scalar

In [36]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
X_train=std_scaler.fit_transform(X_train)
X_test=std_scaler.transform(X_test)

Random Forest

In [37]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

dataframe['rating']=clf.predict(X_test)

Export the test results to Predict_Rating_Logistic_Regression.csv

In [38]:
dataframe.drop(axis="columns",labels=["user_id","book_id","review_text","date_added","date_updated","read_at","started_at","n_votes","n_comments"],inplace=True)

In [39]:
dataframe.to_csv("Predict_Rating_RandomForest.csv",index=False)

In [40]:
d=pd.read_csv("Predict_Rating_RandomForest.csv")
d.head()

,review_id,rating
0,b3d1248da3dd5cb0c15d5e737793ca32,4
1,8ac45a1ad472e40dd19245317087f009,3
2,e083ab7283f9c8f0edd6f5673af348a9,4
3,167e419508a6a188472e9568226da48a,3
4,03a8a7306e3ef4c1ffd95de068b58b4e,4
